In [1]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary

from torchvision import transforms
from torchvision.models import resnet50

import augmentations

In [2]:
print(torch.__version__)

1.7.1


## 1. 커스텀 데이터셋 만들기

In [3]:
def aug(image):
    """
    hyperparms
    """
    mixture_depth = -1
    mixture_width = 3
    aug_severity = 3


    preprocess = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(
          [0.485, 0.456, 0.406],
          [0.229, 0.224, 0.225])])

    aug_list = augmentations.augmentations_all
    ws = np.float32(np.random.dirichlet([1] * mixture_width))
    m = np.float32(np.random.beta(1, 1))

    mix = torch.zeros_like(preprocess(image))
    for i in range(mixture_width):
        image_aug = image.copy()
        depth = mixture_depth if mixture_depth > 0 else np.random.randint(1, 4)
        
        for _ in range(depth):
            op = np.random.choice(aug_list)
            image_aug = op(image_aug, aug_severity)
            # Preprocessing commutes since all coefficients are convex
        
        #print(preprocess(image_aug))
        mix += ws[i] * preprocess(image_aug)

    mixed = (1 - m) * preprocess(image) + m * mix
    return mixed

In [4]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            image = self.transforms(image)

        return aug(image), target

## 2. 이미지 어그멘테이션

In [5]:
transforms_train = transforms.Compose([
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomVerticalFlip(p=0.5),
    # transforms.RandomCrop(32, padding =4), # added random cropping
    #transforms.ToTensor(),
])

transforms_test = transforms.Compose([
    #transforms.ToTensor(),
])

In [6]:
trainset = MnistDataset('2nd_data/dirty_mnist_2nd', '2nd_data/dirty_mnist_2nd_answer.csv', transforms_train)
print('done trainset')
trainset, validset = torch.utils.data.random_split(trainset, [45000,5000])
print('split done')

testset = MnistDataset('2nd_data/test_dirty_mnist_2nd', '2nd_data/sample_submission.csv', transforms_test)
print('done testset')

train_loader = DataLoader(trainset, batch_size=32, num_workers=8)
print('done train loader')

valid_loader = DataLoader(validset, batch_size = 32, num_workers = 4)
print('done valid loader')
test_loader = DataLoader(testset, batch_size=32, num_workers=4)
print('done test laoder')

done trainset
split done
done testset
done train loader
done valid loader
done test laoder


## 3. ResNet50 모형

In [7]:
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = resnet50(pretrained=True)
        self.classifier = nn.Linear(1000, 26)

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)

        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MnistModel().to(device)
print(summary(model, input_size=(1, 3, 256, 256), verbose=0))

Layer (type:depth-idx)                   Output Shape              Param #
├─ResNet: 1-1                            [1, 1000]                 --
|    └─Conv2d: 2-1                       [1, 64, 128, 128]         9,408
|    └─BatchNorm2d: 2-2                  [1, 64, 128, 128]         128
|    └─ReLU: 2-3                         [1, 64, 128, 128]         --
|    └─MaxPool2d: 2-4                    [1, 64, 64, 64]           --
|    └─Sequential: 2-5                   [1, 256, 64, 64]          --
|    |    └─Bottleneck: 3-1              [1, 256, 64, 64]          75,008
|    |    └─Bottleneck: 3-2              [1, 256, 64, 64]          70,400
|    |    └─Bottleneck: 3-3              [1, 256, 64, 64]          70,400
|    └─Sequential: 2-6                   [1, 512, 32, 32]          --
|    |    └─Bottleneck: 3-4              [1, 512, 32, 32]          379,392
|    |    └─Bottleneck: 3-5              [1, 512, 32, 32]          280,064
|    |    └─Bottleneck: 3-6              [1, 512, 32, 32]  

## 4. 학습하기

In [8]:
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [9]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MultiLabelSoftMarginLoss()

num_epochs = 1000

train_losses = []
valid_losses = []
avg_train_losses = []
avg_valid_losses = []

early_stopping = EarlyStopping(patience = 3, verbose = True)

for epoch in range(1,num_epochs+1):
    model.train()
    print('========= Start Epoch {} ========='.format(epoch))
    for i, (images, targets) in enumerate(train_loader):

        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()


        train_losses.append(loss.item())

        if (i+1) % 10 == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            print(f'[{epoch}] loss: {loss.item():.5f}, acc: {acc.item() * 100:.2f}%')


    model.eval()
    for i, (images, targets) in enumerate(valid_loader):
        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        loss = criterion(outputs, targets)

        valid_losses.append(loss.item())

    
    train_loss = np.average(train_losses)
    valid_loss = np.average(valid_losses)
    avg_train_losses.append(train_loss)
    avg_valid_losses.append(valid_loss)

    epoch_len = len(str(num_epochs))

    print_msg = (f'[{epoch:>{epoch_len}}/{num_epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'valid_loss: {valid_loss:.5f}')

    print(print_msg)

    # clear lists to track next epoch
    train_losses = []
    valid_losses = []

    early_stopping(valid_loss, model)

    if early_stopping.early_stop:
        print("Early stopping")
        break

model.load_state_dict(torch.load('checkpoint.pt'))

46, acc: 62.50%
[10] loss: 0.63448, acc: 59.50%
[10] loss: 0.64323, acc: 60.10%
[10] loss: 0.63104, acc: 58.17%
[10] loss: 0.64320, acc: 56.61%
[10] loss: 0.62963, acc: 59.01%
[10] loss: 0.64552, acc: 58.77%
[10] loss: 0.62020, acc: 60.22%
[10] loss: 0.63975, acc: 59.62%
[10] loss: 0.62763, acc: 60.58%
[10] loss: 0.63846, acc: 60.22%
[10] loss: 0.64224, acc: 59.38%
[10] loss: 0.64977, acc: 58.89%
[10] loss: 0.63112, acc: 62.14%
[10] loss: 0.62974, acc: 61.06%
[10] loss: 0.63517, acc: 58.53%
[10] loss: 0.61973, acc: 61.90%
[10] loss: 0.62662, acc: 61.66%
[10] loss: 0.62565, acc: 61.78%
[10] loss: 0.64589, acc: 59.98%
[10] loss: 0.64185, acc: 60.58%
[10] loss: 0.61668, acc: 61.42%
[10] loss: 0.62657, acc: 62.62%
[10] loss: 0.64867, acc: 60.70%
[10] loss: 0.62735, acc: 61.66%
[10] loss: 0.65563, acc: 59.01%
[10] loss: 0.63166, acc: 59.62%
[10] loss: 0.62067, acc: 59.86%
[10] loss: 0.62978, acc: 61.54%
[10] loss: 0.62123, acc: 62.86%
[10] loss: 0.63770, acc: 60.70%
[10] loss: 0.64027, acc:

KeyboardInterrupt: 

## 5. 추론하기

In [ ]:
submit = pd.read_csv('2nd_data/sample_submission.csv')

model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs > 0.5
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()
    
submit.to_csv('submit_augmix.csv', index=False)